<a href="https://colab.research.google.com/github/matheusmf1/aws-bid-machine/blob/main/rhs_dataAnalisys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import json

In [39]:
# fileName = "/content/drive/MyDrive/Colab Notebooks/155917_8_1095_20201207223020.json"
fileName = "/content/drive/MyDrive/Colab Notebooks/155917_3_1091_20201207220101.json"

data = pd.read_json( fileName )

data

,licitacoes
licitacao,[{'palavrasEncontradasAgrupadasXml': {'palavra...


# Prepare dataset

In [40]:
def prepareDataSet( data ):

  df = pd.DataFrame( data )

  # Create new Columns
  createNewColumns( df )

  # Drop nested columns
  df.drop( ['palavrasEncontradasAgrupadasXml', 'linksAgrupadosXml', 'trechosEncontradosAgrupadosXml' ], axis = 1, inplace = True  )

  df.reset_index( drop = True, inplace = True )
  df.drop_duplicates( inplace = True )

  return df

In [41]:
def createNewColumns( df ):
  
  if type( df['palavrasEncontradasAgrupadasXml']['palavra'] ) == str:
    
    df['palavraEncontrada'] = df['palavrasEncontradasAgrupadasXml']['palavra']

  elif type(df['palavrasEncontradasAgrupadasXml']['palavra']) == list:

    df['palavraEncontrada'] = ",".join( df['palavrasEncontradasAgrupadasXml']['palavra'] )  


  if type( df['linksAgrupadosXml']['link'] ) == str:
    
    df['links'] = df['linksAgrupadosXml']['link']

  elif ( type( df['linksAgrupadosXml']['link'] ) == list ):
    
    df['links'] = ",".join( df['linksAgrupadosXml']['link'] )  


  if type( df['trechosEncontradosAgrupadosXml']['trecho'] ) == str:
    
    df['trechos'] = df['trechosEncontradosAgrupadosXml']['trecho']

  elif ( type( df['trechosEncontradosAgrupadosXml']['trecho'] ) == list ):
   
    df['trechos'] = ",".join( df['trechosEncontradosAgrupadosXml']['trecho'] )

In [42]:
licitacaoType = type(data['licitacoes']['licitacao'])

if licitacaoType == dict:
  
  mainData = data['licitacoes']['licitacao']
  df_new = prepareDataSet( mainData ) 

elif licitacaoType == list:

  df_new = pd.DataFrame()
  
  for licitacao in data['licitacoes']['licitacao']:
    df_new = df_new.append( prepareDataSet( licitacao ), ignore_index = True )
    print(df_new['palavraEncontrada'])

0    Armazenamento,AVA,Backup,CRM,EAD,Google,Hardwa...
Name: palavraEncontrada, dtype: object
0    Armazenamento,AVA,Backup,CRM,EAD,Google,Hardwa...
1                   Armazenamento,Cloud,ERP,Nuvem,SAAS
Name: palavraEncontrada, dtype: object
0    Armazenamento,AVA,Backup,CRM,EAD,Google,Hardwa...
1                   Armazenamento,Cloud,ERP,Nuvem,SAAS
2    Armazenamento,Backup,Big Data,Hardware,Microso...
Name: palavraEncontrada, dtype: object


In [29]:
df_new

,objeto,modalidade,registro,orgao,prazoTipo,prazoData,prazoHora,endereco,cidade,uf,telefone,site,palavraEncontrada,links,trechos
0,UASG: 200109 PREGAO ELETRONICO - CONTRATACAO...,PE/21/2020,10235516/11590,Departamento de Policia Rodoviaria Federal,Prazo para Abertura,17/12/2020,09:00,"Spo, Quadra 3, Lote 5 Complexo Sede da PRF, As...",Brasilia,DF,Nao informado pela fonte,None,"Armazenamento,AVA,Backup,CRM,EAD,Google,Hardwa...",https://www.licitacao.com.br/editais/IA-MARE/2...,"originÃ¡rios da contrataÃ§Ã£o, recolhendo-os a..."
1,UASG: 135058 PREGAO ELETRONICO - CONTRATACAO...,PE/14/2020,10246947/11590,EMBRAPA - Empresa Brasileira de Pesquisa Agrop...,Prazo para Abertura das Propostas,18/12/2020,09:00,http://www.comprasnet.gov.br,Brasilia,DF,Nao informado pela fonte,None,"Armazenamento,Cloud,ERP,Nuvem,SAAS",https://www.licitacao.com.br/editais/IA-MARE/2...,"materiais reciclÃ¡veis, de forma a garantir a ..."
2,UASG: 135036 PREGAO ELETRONICO - FORMACAO DE...,PE/38/2020,10246938/11590,EMBRAPA - Empresa Brasileira de Pesquisa Agrop...,Prazo para Abertura das Propostas,17/12/2020,09:00,"Rod. GO 462, Km 12",SANTO ANTONIO DE GOIAS,GO,(62) 3533-2190,None,"Armazenamento,Backup,Big Data,Hardware,Microso...",https://www.licitacao.com.br/editais/IA-MARE/2...,Module) para <em>armazenamento</em> de chaves ...


# Find specific word

In [8]:
jsonBody = df_new.to_json()
jsonBody

'{"objeto":{"0":"UASG: 200109  PREGAO ELETRONICO -  CONTRATACAO DE EMPRESA ESPECIALIZADA NA PRESTACAO DE SERVICOS DE ADMINISTRACAO E GERENCIAMENTO, POR MEIO DE SISTEMA INFORMATIZADO, PARA FORNECIMENTO DE COMBUSTIVEIS, OLEOS, FILTROS LUBRIFICANTES, SERVICOS DE LAVAGENS E DE BORRACHARIA, DE MANUTENCAO PREVENTIVA E CORRETIVA, COM FORNECIMENTO DE PECAS\\/MATERIAIS PARA ATENDER TODAS AS MAQUINAS, EQUIPAMENTOS E VEICULOS DO DEPARTAMENTO DE POLICIA RODOVIARIA E ORGAOS PARTICIPANTES.","1":"UASG: 135058  PREGAO ELETRONICO -  CONTRATACAO DE EMPRESA ESPECIALIZADA PARA GOVERNANCA DE NOVOS CADASTRAMENTOS NA BASE DE DADOS DE MATERIAIS, BENS E SERVICOS; MANUTENCAO DA CONFORMIDADE DOS ITENS DA BASE DE DADOS E;  IMPLANTACAO E CONCESSAO DE USO POR TODA A VIGENCIA DO CONTRATO DE SISTEMA PROPRIO INFORMATIZADO, COMPATIVEL PARA INTEGRACAO AO SISTEMA DE GESTAO INTEGRADO CORPORATIVO (ERP \\/ SAP S\\/4 HANA)","2":"UASG: 135036  PREGAO ELETRONICO -  FORMACAO DE REGISTRO DE PRECO PARA EVENTUAL AQUISICAO DE SERVI

In [9]:
df = pd.read_json( jsonBody )
df

,objeto,modalidade,registro,orgao,prazoTipo,prazoData,prazoHora,endereco,cidade,uf,telefone,site,palavraEncontrada,links,trechos
0,UASG: 200109 PREGAO ELETRONICO - CONTRATACAO...,PE/21/2020,10235516/11590,Departamento de Policia Rodoviaria Federal,Prazo para Abertura,17/12/2020,09:00,"Spo, Quadra 3, Lote 5 Complexo Sede da PRF, As...",Brasilia,DF,Nao informado pela fonte,NaN,"Armazenamento,AVA,Backup,CRM,EAD,Google,Hardwa...",https://www.licitacao.com.br/editais/IA-MARE/2...,"originÃ¡rios da contrataÃ§Ã£o, recolhendo-os a..."
1,UASG: 135058 PREGAO ELETRONICO - CONTRATACAO...,PE/14/2020,10246947/11590,EMBRAPA - Empresa Brasileira de Pesquisa Agrop...,Prazo para Abertura das Propostas,18/12/2020,09:00,http://www.comprasnet.gov.br,Brasilia,DF,Nao informado pela fonte,NaN,"Armazenamento,Cloud,ERP,Nuvem,SAAS",https://www.licitacao.com.br/editais/IA-MARE/2...,"materiais reciclÃ¡veis, de forma a garantir a ..."
2,UASG: 135036 PREGAO ELETRONICO - FORMACAO DE...,PE/38/2020,10246938/11590,EMBRAPA - Empresa Brasileira de Pesquisa Agrop...,Prazo para Abertura das Propostas,17/12/2020,09:00,"Rod. GO 462, Km 12",SANTO ANTONIO DE GOIAS,GO,(62) 3533-2190,NaN,"Armazenamento,Backup,Big Data,Hardware,Microso...",https://www.licitacao.com.br/editais/IA-MARE/2...,Module) para <em>armazenamento</em> de chaves ...


In [10]:
df['palavraEncontrada']

0    Armazenamento,AVA,Backup,CRM,EAD,Google,Hardwa...
1                   Armazenamento,Cloud,ERP,Nuvem,SAAS
2    Armazenamento,Backup,Big Data,Hardware,Microso...
Name: palavraEncontrada, dtype: object

In [11]:
df['palavraEncontrada'] = df['palavraEncontrada'].map( lambda x: x.lower().split(',') )
df['palavraEncontrada']

0    [armazenamento, ava, backup, crm, ead, google,...
1             [armazenamento, cloud, erp, nuvem, saas]
2    [armazenamento, backup, big data, hardware, mi...
Name: palavraEncontrada, dtype: object

In [32]:
from collections import Counter
listWords = [ 'google', 'backup', 'microsoft azure' ]

foundWords = []

for row in df['palavraEncontrada']:
  print(row)
  
  for word in row:
  
    if word in listWords:
      print(f'toop: {word}')
      foundWords.append( word )

print( f'Lista de palavras achada: {foundWords}' )

['armazenamento', 'ava', 'backup', 'crm', 'ead', 'google', 'hardware', 'microsoft']
toop: backup
toop: google
['armazenamento', 'cloud', 'erp', 'nuvem', 'saas']
['armazenamento', 'backup', 'big data', 'hardware', 'microsoft', 'storage', 'vmware']
toop: backup
Lista de palavras achada: ['backup', 'google', 'backup']


In [37]:
number = Counter( foundWords )